In [1]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

/home/spencersong/metamaterials_ai/src


In [2]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
import torch
import numpy as np
import random
import itertools
data_folder = '../local_data'
work_folder = '../local_work'

### Unexplored Params

In [3]:
inc_data = torch.load(
                Path(f"{data_folder}/inconel-revised-raw-shuffled.pt"))["laser_params"]
params = torch.load(
                Path(f"{data_folder}/stainless-steel-revised-shuffled.pt"))["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
max_speed, max_spacing = params.max(0)[0][0].item(), params.max(0)[0][1].item()
min_speed, min_spacing = params.min(0)[0][0].item(), params.min(0)[0][1].item()
print(max_speed, max_spacing)
print(min_speed, min_spacing)

700.0 42.0
10.0 1.0


In [4]:
def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

    watt_arg = torch.argmax(y_hat[0][2:])
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = max_speed * y_hat[0][0] # TODO call the scale
    y_final[1] = max_spacing * y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    return y_final

In [5]:
#identify explored param space
def get_unexplored_params(params, data, max_speed, max_spacing, min_speed, min_spacing, n = 196, seed = 1):
        """
        finds unexplored parameters
        inputs: tensor of [[], [], []], each size (1,14)
        outputs: list of n unnormalized tuples each of size (1,3)
        """
        param_space = [np.arange(min_speed, max_speed + 10, 10).tolist(), np.arange(min_spacing, max_spacing + 1, 1).tolist(), np.around(np.arange(.2, 1.4, .1), decimals = 1).tolist()]
        #get set of all possible parameter combinations
        param_all_comb = set(list(itertools.product(*param_space)))#[list(tup) for tup in itertools.product(*param_space)]
        wattages = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
        #get set of all training parameter combinations -decoding of one hot encoding
        train_param_space = set([tuple(x[:2] + [wattages[np.argmax(x[2:])]]) for x in data.tolist()]) #29988
        #find the combination parameters NOT in the training space
        unused_params = list(param_all_comb.symmetric_difference(train_param_space)) #len() = 5292
        #sample n from unused parameter combinations
        random.seed = seed
        indices = random.sample(range(len(unused_params)), n)
        return [unused_params[i] for i in indices]

def unexplored_params_to_tensors(x):
        new_x = []
        wattage_idxs = {
                0.2: 0,
                0.3: 1,
                0.4: 2,
                0.5: 3,
                0.6: 4,
                0.7: 5,
                0.8: 6,
                0.9: 7,
                1.0: 8,
                1.1: 9,
                1.2: 10,
                1.3: 11,
        }      
        for i in x:
                i = list(i)
                holder = i[:2] + np.zeros(12).tolist()
                holder[wattage_idxs[i[2]] + 2] = 1.0
                new_x.append(holder)

        laser_params = torch.FloatTensor(new_x)
        for r in laser_params:
                r[0]=r[0] / max_speed
                r[1]=r[1] / max_spacing
        return laser_params

#[torch.cat((x[:2], torch.tensor(np.array([wattages[np.argmax(x[2:])]])))) for x in params]#[:5]
#[x[:2] for x in params]#[:5]

# train_param_space = denormalize_decode_result(params, max_speed, max_spacing, min_speed, min_spacing)
# train_param_space


In [6]:
ouput_unexplored = get_unexplored_params(params, inc_data, max_speed, max_spacing, min_speed, min_spacing, n = 1200, seed = 1)
norm_unexplored_laser_params = unexplored_params_to_tensors(ouput_unexplored)

In [7]:
norm_unexplored_laser_params

tensor([[0.8714, 0.2857, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0286, 0.1429, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.6143, 0.8571, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        ...,
        [0.5143, 0.3333, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.1571, 0.1905, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.8714, 0.0714, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

# Inverse Model Only


In [14]:
from src.models import InverseModel

In [92]:
filepath = Path(
    f"{work_folder}/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt") #CHANGEME inconel vs stainless best model
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt


In [93]:
i_model = InverseModel.load_from_checkpoint(filepath, strict=False)
i_model.eval()

x = torch.from_numpy(np.array(torch.randn(1, 800) * 3)) #
with torch.no_grad():
    y_hat = i_model(x)
y_hat.shape

torch.Size([1, 14])

In [94]:
# y_final2 = onehot_to_continuous2(y_hat)
print(denormalize_decode_result(y_hat,max_speed, max_spacing, min_speed, min_spacing))

#if you want hardcoded max, min values instead
#print(denormalize_decode_result(y_hat,700.0, 42.0, 10.0, 1.0))

tensor([ 4.7735, 36.7123,  1.1000])


# Direct Model Only


In [8]:
from src.models import DirectModel

In [9]:
filepath = Path(
    f"{work_folder}/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt")
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt


In [10]:
d_model = DirectModel.load_from_checkpoint(filepath)
d_model.eval()
# x = torch.randn(1, 14)
direct_y_hats = []
# intersect_preds = []
for i in norm_unexplored_laser_params:
    with torch.no_grad():
        direct_y_hats.append(d_model(i.reshape(1,14)))

### Unexplored Direct --> Indirect

In [15]:
filepath = Path(
    f"{work_folder}/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt") #CHANGEME inconel vs stainless best model
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt


In [16]:
model = InverseModel.load_from_checkpoint(filepath, strict=False)
model.eval()

#x = torch.from_numpy(np.array(torch.ra|ndn(1, 800) * 3)) #
param_preds = []
for i in direct_y_hats:
    with torch.no_grad():
        param_preds.append(model(i))#model(i))

/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:217: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['direct_model.model.0.weight', 'direct_model.model.0.bias', 'direct_model.model.2.layers.0.block.0.weight', 'direct_model.model.2.layers.0.block.0.bias', 'direct_model.model.2.layers.1.block.0.weight', 'direct_model.model.2.layers.1.block.0.bias', 'direct_model.model.2.layers.2.block.0.weight', 'direct_model.model.2.layers.2.block.0.bias', 'direct_model.model.2.layers.3.block.0.weight', 'direct_model.model.2.layers.3.block.0.bias', 'direct_model.model.2.layers.4.block.0.weight', 'direct_model.model.2.

In [17]:
params = torch.load(
                Path(f"{data_folder}/inconel-revised-raw-shuffled.pt"))["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
max_speed, max_spacing = params.max(0)[0][0].item(), params.max(0)[0][1].item()
min_speed, min_spacing = params.min(0)[0][0].item(), params.min(0)[0][1].item()
decoded_preds = [denormalize_decode_result(pred, max_speed, max_spacing, min_speed, min_spacing) for pred in param_preds]
for r in decoded_preds:
    r[0]=torch.round(r[0], decimals = -1)
    r[1]=torch.round(r[1], decimals = 0)

In [18]:
param_space = [np.arange(min_speed, max_speed + 10, 10).tolist(), np.arange(min_spacing, max_spacing + 1, 1).tolist(), np.around(np.arange(.2, 1.4, .1), decimals = 1).tolist()]
#get set of all possible parameter combinations
param_all_comb = set(list(itertools.product(*param_space)))
wattages = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
space_params = set([tuple(np.around(x.tolist(), decimals= 1)) for x in decoded_preds]) #29988
        #find the combination parameters NOT in the training space
overlaps = list(param_all_comb.intersection(space_params)) #len() = 5292
unused_params = list(set(overlaps).symmetric_difference(space_params)) #len() = 5292
#convert preds to set
print(len(space_params))
print(len(overlaps)) #332
len(unused_params) #216



582
349


233

In [22]:
#send these to minok
unused_params[:196]#overlaps)
sorted(unused_params[:196], key = lambda x: (x[2], x[0], x[1]))
#overlaps[:195]


[(670.0, 9.0, 0.5),
 (670.0, 11.0, 0.5),
 (670.0, 12.0, 0.5),
 (0.0, 28.0, 0.8),
 (80.0, 13.0, 1.2),
 (90.0, 14.0, 1.2),
 (100.0, 11.0, 1.2),
 (100.0, 13.0, 1.2),
 (110.0, 11.0, 1.2),
 (110.0, 12.0, 1.2),
 (110.0, 14.0, 1.2),
 (120.0, 11.0, 1.2),
 (120.0, 13.0, 1.2),
 (140.0, 9.0, 1.2),
 (140.0, 10.0, 1.2),
 (140.0, 12.0, 1.2),
 (140.0, 14.0, 1.2),
 (150.0, 13.0, 1.2),
 (160.0, 12.0, 1.2),
 (160.0, 13.0, 1.2),
 (160.0, 14.0, 1.2),
 (180.0, 9.0, 1.2),
 (180.0, 11.0, 1.2),
 (180.0, 13.0, 1.2),
 (200.0, 11.0, 1.2),
 (200.0, 14.0, 1.2),
 (210.0, 8.0, 1.2),
 (210.0, 10.0, 1.2),
 (210.0, 13.0, 1.2),
 (220.0, 12.0, 1.2),
 (230.0, 13.0, 1.2),
 (230.0, 14.0, 1.2),
 (240.0, 12.0, 1.2),
 (240.0, 13.0, 1.2),
 (240.0, 14.0, 1.2),
 (250.0, 10.0, 1.2),
 (250.0, 11.0, 1.2),
 (250.0, 12.0, 1.2),
 (260.0, 11.0, 1.2),
 (260.0, 12.0, 1.2),
 (260.0, 14.0, 1.2),
 (270.0, 8.0, 1.2),
 (270.0, 9.0, 1.2),
 (270.0, 10.0, 1.2),
 (270.0, 11.0, 1.2),
 (270.0, 13.0, 1.2),
 (280.0, 8.0, 1.2),
 (280.0, 11.0, 1.2),
 (2

In [256]:
#intersecting parameters!!!!!!!
intersect_x = []
wattage_idxs = {
        0.2: 0,
        0.3: 1,
        0.4: 2,
        0.5: 3,
        0.6: 4,
        0.7: 5,
        0.8: 6,
        0.9: 7,
        1.0: 8,
        1.1: 9,
        1.2: 10,
        1.3: 11,
}      
for i in unused_params:
        i = list(i)
        holder = i[:2] + np.zeros(12).tolist()
        holder[wattage_idxs[i[2]] + 2] = 1.0
        intersect_x.append(holder)
output = torch.tensor(intersect_x)
output

tensor([[600.,  34.,   0.,  ...,   0.,   0.,   0.],
        [590.,  14.,   0.,  ...,   0.,   1.,   0.],
        [580.,  31.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [640.,  16.,   0.,  ...,   0.,   1.,   0.],
        [520.,  38.,   0.,  ...,   0.,   0.,   0.],
        [300.,  37.,   0.,  ...,   0.,   0.,   0.]], dtype=torch.float64)

In [136]:
for i in range(len(params)):
    curr = params[i].flatten().tolist()
    if curr in output.tolist():
        print(i)

12058
19562
19609


In [149]:
from utils import rmse
# params[12058]
data = torch.load(
                Path(f"{data_folder}/stainless-steel-revised-shuffled.pt"))

intersect_train_emiss = []
intersect_train_emiss.append(data["emissivity"][12058])
intersect_train_emiss.append(data["emissivity"][19562])
intersect_train_emiss.append(data["emissivity"][19609])

# ONNX

In [ ]:
# import onnxruntime 
# from pathlib import Path
# import numpy as np
# import os

In [ ]:
ort_session = onnxruntime.InferenceSession(filepath)

In [ ]:
# filepath = Path(f"{work_folder}/saved_best/best_inference_direct.onnx")

# ort_session = onnxruntime.InferenceSession(filepath)
# input_name = ort_session.get_inputs()[0].name
# ort_inputs = {input_name: np.random.randn(2, 14)}
# ort_outs = ort_session.run(None, ort_inputs)
